In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import time 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import metrics

In [3]:
train_all = pd.read_csv('train1090.csv')
#train = pd.read_csv('train1040.csv')
#train = pd.read_csv('train100900.csv')
#train_all = pd.read_csv('train005995.csv')

In [4]:
train = train.sort_values(by=['index']).reset_index()
train.head(5)

,level_0,Unnamed: 0,index,click,weekday,hour,region,city,adexchange,slotwidth,...,browser_other,browser_safari,browser_sogou,browser_theworld,bidprice,payprice,counts_c,freq_c,counts_r,freq_r
0,1907,314,314,0,0,9,146,162,3,960,...,0,0,0,0,300,50,5081,0.002090,160464,0.066008
1,3234,579,579,0,4,18,3,4,1,950,...,0,0,0,0,238,2,23899,0.009831,108509,0.044636
2,5886,1069,1069,0,0,14,3,5,3,960,...,0,0,0,0,300,20,11572,0.004760,108509,0.044636
3,7172,1104,1104,1,1,19,79,79,3,1000,...,0,0,0,0,300,80,63513,0.026126,63513,0.026126
4,4143,1547,1547,0,2,12,368,373,2,728,...,0,0,0,0,300,184,723,0.000297,8991,0.003699


In [6]:
train = train.drop(train.columns[[0,1]],axis =1)
#test= test.drop(test.columns[[0]],axis = 1)
train.head(5)

,index,click,weekday,hour,region,city,adexchange,slotwidth,slotheight,slotprice,...,browser_other,browser_safari,browser_sogou,browser_theworld,bidprice,payprice,counts_c,freq_c,counts_r,freq_r
0,314,0,0,9,146,162,3,960,90,50,...,0,0,0,0,300,50,5081,0.002090,160464,0.066008
1,579,0,4,18,3,4,1,950,90,0,...,0,0,0,0,238,2,23899,0.009831,108509,0.044636
2,1069,0,0,14,3,5,3,960,90,20,...,0,0,0,0,300,20,11572,0.004760,108509,0.044636
3,1104,1,1,19,79,79,3,1000,90,80,...,0,0,0,0,300,80,63513,0.026126,63513,0.026126
4,1547,0,2,12,368,373,2,728,90,5,...,0,0,0,0,300,184,723,0.000297,8991,0.003699


In [14]:
#100 features
y_ctr = train['click']
x_ctr = train.drop(['index','region','city','counts_r','counts_c','click','payprice','bidprice'],axis =1)
print(y_ctr.shape)
print(x_ctr.shape)

(8965,)
(8965, 100)


In [15]:
train_x, test_x, train_y, test_y = train_test_split(x_ctr, y_ctr, test_size=0.2)
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

(7172, 100)
(7172,)
(1793, 100)
(1793,)


## Testing out different models

In [16]:
#"MODEL 1 - Decision Tree Classifier"

start = time.time()
clf = DecisionTreeClassifier(max_depth = 15, random_state = 0)
clf.fit(train_x, train_y)
prediction1 = np.array(clf.predict(test_x))

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction1)
auc = metrics.auc(fpr, tpr)

score = accuracy_score(test_y, prediction1)
end = time.time()

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m1 = clf.predict_proba(test_x)  
    
print("MODEL 1 - Decision Tree Classifier")
print("score =", score)
print("auc score =", auc)
print ("error_model_1 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
print(pCTR_m1[:,1])
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]




MODEL 1 - Decision Tree Classifier
score = 0.846625766871
auc score = 0.743342814114
error_model_1 =  0.153374233129
Running time = 0.053954124450683594
pCTR: 
[ 0.08076923  0.          0.7037037  ...,  0.          0.          0.        ]


In [34]:
r_pctr=pd.DataFrame(data =results, columns=['pCTR'])
r.head(5)
r.shape

(1793, 1)

In [17]:
#"MODEL 2 - Extra Tree Classifier"

start = time.time()
clf = ExtraTreesClassifier(max_depth = 15, random_state=0)
#class_weight={0:0.8,1:0.2}, max_depth = 10, n_estimators=10,min_samples_leaf=10, 
clf.fit(train_x, train_y)
prediction2 = np.array(clf.predict(test_x))
score = accuracy_score(test_y, prediction2)
end = time.time()

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction2)
auc = metrics.auc(fpr, tpr)

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m2 = clf.predict_proba(test_x)  
    
print("MODEL 2 - Extra Tree Classifier")
print("score =", score)
print("auc score =", auc)
print ("error_model_2 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
print(pCTR_m2[:,1])
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]


MODEL 2 - Extra Tree Classifier
score = 0.876185164529
auc score = 0.726471152194
error_model_2 =  0.123814835471
Running time = 0.07992911338806152
pCTR: 
[ 0.13458065  0.04935949  0.47238114 ...,  0.05053282  0.10428407
  0.09057756]


In [18]:
#"MODEL 3 - Random Forest Classifier"

start = time.time()
clf = RandomForestClassifier(max_depth = 15)
#min_samples_leaf=20
clf.fit(train_x, train_y)
prediction3 = np.array(clf.predict(test_x))
score = accuracy_score(test_y, prediction3)

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction3)
auc = metrics.auc(fpr, tpr)

end = time.time()

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m3 = clf.predict_proba(test_x)  
    
print("MODEL 3 - Random Forest Classifier")
print("score =", score)
print("auc score =", auc)
print ("error_model_3 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
print(pCTR_m3[:,1])
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]


MODEL 3 - Random Forest Classifier
score = 0.87228109314
auc score = 0.713056912648
error_model_3 =  0.12771890686
Running time = 0.07909703254699707
pCTR: 
[ 0.28027405  0.0601844   0.47911117 ...,  0.02190687  0.082931    0.15446418]


In [19]:
#"MODEL 4 - Adaboost Classifier"

start = time.time()
clf_base = DecisionTreeClassifier()
clf = AdaBoostClassifier(n_estimators=50, base_estimator=clf_base,learning_rate=0.5)
clf.fit(train_x, train_y)
prediction4 = np.array(clf.predict(test_x))
score = accuracy_score(test_y, prediction4)

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction4)
auc = metrics.auc(fpr, tpr)

end = time.time()

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m4 = clf.predict_proba(test_x)  
    
print("MODEL 4 - Adaboost Classifier")
print("score =", score)
print("auc score =", auc)
print ("error_model_4 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]
pCTR_1_m4 = pCTR_m4[:,1]
print(pCTR_1_m4)


MODEL 4 - Adaboost Classifier
score = 0.817066369214
auc score = 0.725029061523
error_model_4 =  0.182933630786
Running time = 0.7671270370483398
pCTR: 
[ 0.0250941   0.05013313  0.18827188 ...,  0.02517506  0.04887171
  0.04934203]


In [20]:
#"MODEL 6 - Logistic Regression Classifier"

start = time.time()
clf = LogisticRegression()
clf.fit(train_x, train_y)
prediction6 = np.array(clf.predict(test_x))
score = accuracy_score(test_y, prediction6)
end = time.time()

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction6)
auc = metrics.auc(fpr, tpr)

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m6 = clf.predict_proba(test_x)  
    
print("MODEL 6 - Logistic Classifier")
print("score =", score)
print("auc =", auc)
print ("error_model_6 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
print(pCTR_m6[:,1])
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]


MODEL 6 - Logistic Classifier
score = 0.87228109314
auc = 0.718554633665
error_model_6 =  0.12771890686
Running time = 0.26584506034851074
pCTR: 
[ 0.02788685  0.09790855  0.52887312 ...,  0.02149759  0.12991526
  0.08619453]


In [21]:
#"MODEL 7 - Xgboost Classifier"

from xgboost import XGBClassifier

start = time.time()
clf = XGBClassifier()
clf.fit(train_x, train_y)
prediction7 = np.array(clf.predict(test_x))

fpr, tpr, thresholds = metrics.roc_curve(test_y, prediction7)
auc = metrics.auc(fpr, tpr)

score = accuracy_score(test_y, prediction7)
end = time.time()

#for row in test_x.iterrows():
#    test_x['pCTR'] = clf.predict_proba(test_x[row])
pCTR_m7 = clf.predict_proba(test_x)  
    
print("MODEL 7 - Xgboost Classifier")
print("score =", score)
print("auc =", auc)
print ("error_model_7 = ", 1-score)
print("Running time =", end - start)
print("pCTR: ")
print(pCTR_m7[:,1])
#print(pCTR_m1[1],axis =1)
#test_x.assign('pCTR')
#test_x['pCTR'] = pCTR_m1[1]


MODEL 7 - Xgboost Classifier
score = 0.887339654211
auc = 0.738879723246
error_model_7 =  0.112660345789
Running time = 1.1500837802886963
pCTR: 
[ 0.05912987  0.07069881  0.52397525 ...,  0.03444372  0.07563879
  0.04898234]


In [45]:
result_m7 = pd.DataFrame(data = pCTR_m7[:,1], columns=['pCTR'])
result_m7.head(5)


,pCTR
0,0.059130
1,0.070699
2,0.523975
3,0.090823
4,0.131108
